<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=99505f34fdfaee393acb1b7ab3e0148707762ab204fc1ab85c55d6aa32dc419f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-05 10:40:35
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: -11.75 K (-0.08%)
Current PnL: -18.25 L (-12.28%)
CY Booked + Current PnL: -8.39 L (-5.65%)
-------------------
Total profit:  2.91 L
Total loss:  -21.16 L
-------------------
Total Booked + Current PnL: 18.59 L (15.27%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.05%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.96 L (56.48%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.50,71.0,H-MC,7.44,143048.0,28112.0,8468.0,0.16,24.46,5.92,31.83,81.0,3.32,1.08,48.27,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-13.20,63.0,H-LC,10.10,173492.0,17315.0,8779.0,-0.18,11.09,5.06,16.71,22.0,1.97,1.31,30.46,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,105.55,66.0,M-SC,6.36,88536.0,-12241.0,12333.0,0.06,-12.15,13.93,0.09,245.0,-0.99,0.67,16.32,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,47.0,M-LC,4.50,214014.0,12534.0,14681.0,-0.22,6.22,6.86,13.51,60.0,0.85,1.61,19.60,XY25,NTT,BANKS
26,DMART,5201.00,-4.91,73.0,H-LC,15.53,237449.0,52968.0,17405.0,0.34,28.71,7.33,38.14,38.0,3.04,1.79,42.36,X40N,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.11,38.0,H-MC,2.19,135770.0,-18.0,42768.0,0.16,-0.01,31.50,31.48,107.0,-0.00,1.02,12.10,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.94,58.0,H-MC,1.93,245513.0,-2218.0,77287.0,-0.69,-0.90,31.48,30.30,92.0,-0.03,1.85,12.28,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.11,43.0,H-MC,4.15,137307.0,1174.0,29947.0,-0.85,0.86,21.81,22.86,91.0,0.04,1.04,13.58,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.02,59.0,H-MC,4.98,202605.0,1785.0,66981.0,-0.13,0.89,33.06,34.25,80.0,0.03,1.53,6.13,X40,ATH,FMCG
88,WIPRO,420.00,-13.79,41.0,M-LC,6.95,151460.0,515.0,108945.0,-0.28,0.34,71.93,72.51,53.0,0.00,1.14,6.38,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-1.51,34.0,H-LC,9.95,224432.0,-17492.0,80773.0,-0.96,-7.23,35.99,26.15,8.0,-0.22,1.69,6.06,X40,ATH,IT
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307977.0,-4911.0,176594.0,-1.17,-1.57,57.34,54.87,3.0,-0.03,2.32,4.98,X40,BTT,IT
45,JIOFIN,387.00,-2.04,39.0,H-LC,13.94,216615.0,-791.0,54284.0,-1.35,-0.36,25.06,24.61,48.0,-0.01,1.63,54.25,XY24,BTT,FINANCE
85,VBL,671.64,-14.77,40.0,H-LC,6.90,307197.0,-8645.0,121312.0,-1.63,-2.74,39.49,35.67,5.0,-0.07,2.32,10.74,X40N,ATH,FMCG
76,TCS,4998.00,-28.04,42.0,H-LC,14.38,287208.0,-58748.0,182607.0,-1.30,-16.98,63.58,35.80,1.0,-0.32,2.17,1.74,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-6.07,51.0,H-LC,1.93,233122.0,-13029.0,92246.0,-0.57,-5.29,39.57,32.18,16.0,-0.14,1.76,28.84,X200,ATH,IT
52,NESTLEIND,1377.00,-8.60,60.0,H-LC,2.54,281128.0,15702.0,41101.0,-0.92,5.92,14.62,21.40,11.0,0.38,2.12,12.95,XY25,NTT,FMCG
43,ITC,452.00,-38.34,48.0,H-LC,3.06,199096.0,-1042.0,21940.0,-2.10,-0.52,11.02,10.44,4.0,-0.05,1.50,5.01,X40,NTT,FMCG
19,CIPLA,1795.00,-19.09,57.0,H-LC,5.60,215761.0,11261.0,30163.0,-0.18,5.51,13.98,20.25,10.0,0.37,1.63,14.72,X40N,BTT,PHARMA
85,VBL,671.64,-14.77,40.0,H-LC,6.90,307197.0,-8645.0,121312.0,-1.63,-2.74,39.49,35.67,5.0,-0.07,2.32,10.74,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SHALBY,327.00,970.18,65.0,M-SC,3.38,145558.0,-35585.0,81047.0,7.04,-19.64,55.68,25.10,235.0,-0.44,1.10,16.65,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3997.30,79.0,M-SC,9.60,127688.0,-19680.0,71493.0,6.28,-13.35,55.99,35.16,236.0,-0.28,0.96,30.42,XY24,NTT,AUTO
69,SONACOMS,1006.00,-37.48,46.0,M-SC,12.72,77866.0,-23293.0,98181.0,1.60,-23.03,126.09,74.03,202.0,-0.24,0.59,9.80,AR,BTT,AUTO
87,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,4.22,102780.0,11282.0,67475.0,1.56,12.33,65.65,86.07,223.0,0.17,0.78,49.26,XR,NTT,DURABLES
86,VOLTAS,1918.49,1.72,63.0,H-MC,4.00,213495.0,21753.0,74275.0,1.48,11.34,34.79,50.08,99.0,0.29,1.61,18.91,XY25,ATH,AC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SIS,528.00,2181.35,44.0,H-SC,5.69,61378.0,-19712.0,28381.0,-2.38,-24.31,46.24,10.69,156.0,-0.69,0.46,24.39,OX40N,NTT,MISC
43,ITC,452.00,-38.34,48.0,H-LC,3.06,199096.0,-1042.0,21940.0,-2.10,-0.52,11.02,10.44,4.0,-0.05,1.50,5.01,X40,NTT,FMCG
85,VBL,671.64,-14.77,40.0,H-LC,6.90,307197.0,-8645.0,121312.0,-1.63,-2.74,39.49,35.67,5.0,-0.07,2.32,10.74,X40N,ATH,FMCG
63,SATIN,274.00,-29.04,52.0,H-SC,8.34,141431.0,-54543.0,127359.0,-1.60,-27.83,90.05,37.16,126.0,-0.43,1.07,7.53,XY24,NTT,FINANCE
45,JIOFIN,387.00,-2.04,39.0,H-LC,13.94,216615.0,-791.0,54284.0,-1.35,-0.36,25.06,24.61,48.0,-0.01,1.63,54.25,XY24,BTT,FINANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,105.55,66.0,M-SC,6.36,88536.0,-12241.0,12333.0,0.06,-12.15,13.93,0.09,245.0,-0.99,0.67,16.32,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,83.20,43.0,H-SC,13.19,115614.0,-26865.0,46384.0,-0.67,-18.86,40.12,13.70,163.0,-0.58,0.87,36.46,OX40N,NTT,BANKS
46,KANSAINER,340.0,-68.34,54.0,H-SC,3.49,218439.0,-51228.0,87550.0,-0.22,-19.00,40.08,13.47,138.0,-0.59,1.65,10.48,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,115.40,51.0,M-SC,3.54,141211.0,-33348.0,33382.0,-1.21,-19.10,23.64,0.02,221.0,-1.00,1.07,22.83,OX40N,NTT,PAINTS
18,CERA,9475.0,-20.05,45.0,H-SC,2.77,114057.0,-29746.0,56492.0,-0.45,-20.69,49.53,18.60,149.0,-0.53,0.86,25.87,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.04,42.0,H-LC,14.38,287208.0,-58748.0,182607.0,-1.30,-16.98,63.58,35.80,1.0,-0.32,2.17,1.74,X40,BTT,IT
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307977.0,-4911.0,176594.0,-1.17,-1.57,57.34,54.87,3.0,-0.03,2.32,4.98,X40,BTT,IT
43,ITC,452.00,-38.34,48.0,H-LC,3.06,199096.0,-1042.0,21940.0,-2.10,-0.52,11.02,10.44,4.0,-0.05,1.50,5.01,X40,NTT,FMCG
85,VBL,671.64,-14.77,40.0,H-LC,6.90,307197.0,-8645.0,121312.0,-1.63,-2.74,39.49,35.67,5.0,-0.07,2.32,10.74,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,44.0,H-LC,13.12,245582.0,-16037.0,135242.0,-0.90,-6.13,55.07,45.57,7.0,-0.12,1.85,4.50,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187775.0,36862.0,20449.0,0.26,24.43,10.89,37.97,93.0,1.80,1.42,55.91,AR,ATH,AUTO
25,DIXON,18940.15,-28.50,71.0,H-MC,7.44,143048.0,28112.0,8468.0,0.16,24.46,5.92,31.83,81.0,3.32,1.08,48.27,X40N,ATH,IT
26,DMART,5201.00,-4.91,73.0,H-LC,15.53,237449.0,52968.0,17405.0,0.34,28.71,7.33,38.14,38.0,3.04,1.79,42.36,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,12.23,62.0,H-MC,3.36,193830.0,29160.0,38805.0,-0.29,17.71,20.02,41.28,89.0,0.75,1.46,27.10,X40N,ATH,AC
86,VOLTAS,1918.49,1.72,63.0,H-MC,4.00,213495.0,21753.0,74275.0,1.48,11.34,34.79,50.08,99.0,0.29,1.61,18.91,XY25,ATH,AC
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187775.0,36862.0,20449.0,0.26,24.43,10.89,37.97,93.0,1.80,1.42,55.91,AR,ATH,AUTO
23,DABUR,735.00,-1.31,66.0,H-MC,4.46,214734.0,17706.0,71914.0,-0.38,8.99,33.49,45.49,102.0,0.25,1.62,23.20,XY24,BTT,FMCG
17,CAMS,5211.76,-5.83,49.0,H-SC,5.49,106960.0,4956.0,38966.0,1.47,4.86,36.43,43.06,122.0,0.13,0.81,22.81,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187775.0,36862.0,20449.0,0.26,24.43,10.89,37.97,93.0,1.80,1.42,55.91,AR,ATH,AUTO
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.56,110282.0,9778.0,20865.0,0.68,9.73,18.92,30.49,79.0,0.47,0.83,45.29,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,4.22,102780.0,11282.0,67475.0,1.56,12.33,65.65,86.07,223.0,0.17,0.78,49.26,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.65,45.0,H-SC,12.22,135012.0,11676.0,122091.0,-0.54,9.47,90.43,108.45,119.0,0.10,1.02,33.94,AR,ATH,MISC
88,WIPRO,420.00,-13.79,41.0,M-LC,6.95,151460.0,515.0,108945.0,-0.28,0.34,71.93,72.51,53.0,0.00,1.14,6.38,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,6.10,83190.0,-19020.0,112406.0,1.32,-18.61,135.12,91.37,208.0,-0.17,0.63,35.04,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-39.64,61.0,L-SC,36.90,88512.0,-25037.0,65092.0,-0.26,-22.05,73.54,35.27,268.0,-0.38,0.67,116.64,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-3.83,46.0,H-SC,15.65,89479.0,-11484.0,106015.0,-0.87,-11.37,118.48,93.63,148.0,-0.11,0.68,33.28,SR,ATH,CHEMICALS
50,MASFIN,398.61,-18.70,48.0,H-SC,13.26,92250.0,-5730.0,27334.0,-1.01,-5.85,29.63,22.05,152.0,-0.21,0.70,34.09,XR,ATH,FINANCE
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.56,110282.0,9778.0,20865.0,0.68,9.73,18.92,30.49,79.0,0.47,0.83,45.29,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,61.0,L-SC,36.90,88512.0,-25037.0,65092.0,-0.26,-22.05,73.54,35.27,268.0,-0.38,0.67,116.64,XR,NTT,HOTELS
7,ATULAUTO,844.00,3997.30,79.0,M-SC,9.60,127688.0,-19680.0,71493.0,6.28,-13.35,55.99,35.16,236.0,-0.28,0.96,30.42,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187775.0,36862.0,20449.0,0.26,24.43,10.89,37.97,93.0,1.80,1.42,55.91,AR,ATH,AUTO
57,RELAXO,1176.00,-31.65,61.0,H-SC,4.98,63405.0,-52133.0,87125.0,-0.59,-45.12,137.41,30.28,136.0,-0.60,0.48,24.65,X40N,NTT,FOOTWEAR
26,DMART,5201.00,-4.91,73.0,H-LC,15.53,237449.0,52968.0,17405.0,0.34,28.71,7.33,38.14,38.0,3.04,1.79,42.36,X40N,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.80
1,25,43.73
2,50,73.89


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.99
LC    36.20
MC    25.83
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.82
X40      15.62
X40N     14.41
XY25     10.10
AR        9.51
XR        9.06
OX40N     7.44
X200      1.76
MH        1.75
X5K       1.44
SR        1.11
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.93
H-MC    22.84
H-SC    22.60
M-SC    13.81
M-LC     7.18
M-MC     2.67
L-SC     1.58
L-LC     1.09
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.95,0.28,33.95
IT,13.41,-16.15,73.75
FINANCE,7.40,-23.18,68.60
BANKS,7.37,-16.62,66.14
PAINTS,6.09,-14.68,40.35
MISC,5.86,-18.38,86.23
ELECTRICAL,5.71,-8.68,68.58
AUTO,4.87,-4.70,50.26
AC,3.80,4.03,28.46


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2780656.0,22
AR,1219571.0,10
XR,1093260.0,13
X40,896350.0,10
X40N,523940.0,11
OX40N,472715.0,10
XY25,411349.0,7
SR,196842.0,2
X5K,126762.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2782439.0,27
M-SC,1418795.0,17
H-LC,1417299.0,16
H-MC,1177434.0,17
M-LC,425657.0,5
M-MC,324380.0,2
L-SC,251416.0,3
L-MC,59658.0,1
L-LC,39434.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,934530.0,6
M-SC,XY24,835231.0,7
H-SC,AR,662725.0,5
H-LC,X40,608480.0,5
H-SC,XR,526594.0,6
H-MC,XY24,456962.0,4
H-LC,AR,376167.0,2
M-MC,XY24,324380.0,2
H-LC,X40N,219379.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
